# THIS IS THE CODE  THAT CREATED THE FEATURES DATASET FROM THE PRE-TRAINED MODELS
## IT ASSUMES THE EXISTANCE OF THE FOLDER ' the_dataset/train' which contains 1000 the images from each malware family

## it also assumes that there is a folder with pre-trained models 


# its output is the folder named 'features' where it will contain only 100 tensor files for each folder

In [4]:
from torchvision import models
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import os
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import DataLoader,Dataset
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import skimage.io as io
from torchsummary import summary
from tqdm import tqdm

In [3]:
dataset='../project/the_dataset/train'
pretrained_models='./pretrained models/'
model_names=['vgg16','resnet50','inception_v3', 'squeezenet']

model_dict={}

target_root='./features'

# Loading the models

In [20]:
vgg16= models.vgg16()
vgg16.load_state_dict(torch.load(pretrained_models+'vgg16.pth'))
vgg16.eval()


resnet50=models.resnet50()
resnet50.load_state_dict(torch.load(pretrained_models+'resnet50.pth'))
resnet50.eval()

inception_v3=models.inception_v3()
inception_v3.load_state_dict(torch.load(pretrained_models+'inception_v3.pth'))
inception_v3.eval()

squeezenet=models.squeezenet1_0()
squeezenet.load_state_dict(torch.load(pretrained_models+'squeezenet.pth'))
squeezenet.eval()


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

# Building feature extractors from the models

In [39]:
vgg16_extractor=torch.nn.Sequential(vgg16.features, vgg16.avgpool)
vgg16_extractor=torch.nn.Sequential(vgg16_extractor, nn.Flatten())

resnet50.fc=nn.Flatten()
resnet50_extractor=resnet50    

inception_v3.fc = nn.Flatten()
inception_v3_extractor=inception_v3


squeezenet_extactor= squeezenet.features
squeezenet_extactor=torch.nn.Sequential(squeezenet_extactor,torch.nn.Flatten())

feature_extractors=[vgg16_extractor,resnet50_extractor, inception_v3_extractor,squeezenet_extactor]

In [40]:
feature_extractors_dict = dict(zip(model_names, feature_extractors))

In [23]:
transform = transforms.Compose([        # Defining a variable transforms
 transforms.Resize(224),                # Resize the image to 256×256 pixels
 transforms.ToTensor(),                 # Convert the image to PyTorch Tensor data type
 transforms.Normalize(                  # Normalize the image
 mean=[0.485, 0.456, 0.406],            # Mean and std of image as also used when training the network
 std=[0.229, 0.224, 0.225]      
 )])

In [80]:
for (root,dirs,files) in  os.walk(dataset, topdown=True):
    if len(files)== 0:
        continue

    if 'valid' in root:
        continue
        
    dirname = root.split(os.path.sep)[-1]
    print("Now in folder", root)
    
    for name in model_names:
        os.mkdir(target_root+'/'+name+'/'+dirname)
    

    for i in tqdm(files[:100]):
        
        if i[-3:] != 'png':
            continue
        img=Image.open(root+'/'+i)
        
        # channel
        img=np.stack((img,)*3, axis=-1)  # stack 
        img=Image.fromarray(img)
        
        img_t=transform(img) # tensorize
        
        img_U= torch.unsqueeze(img_t,0) # 
        
        
        for key in feature_extractors_dict:
            features[key]=feature_extractors_dict[key](img_U)
            
            
        for feature in features:
            target_path=target_root+'/'+feature+'/'+dirname+'/'+i[:-4]+'.pt'
            print("gonna save " , target_path)
#             here=here
            torch.save(features[feature], target_path)


Now in folder ../project/the_dataset/train/Allaple.A


  1%|█                                                                                                          | 1/100 [00:00<00:58,  1.68it/s]

gonna save  ./features/vgg16/Allaple.A/00f971e69aae4ccd38f7edae05aa5314.pt
gonna save  ./features/resnet50/Allaple.A/00f971e69aae4ccd38f7edae05aa5314.pt
gonna save  ./features/inception_v3/Allaple.A/00f971e69aae4ccd38f7edae05aa5314.pt
gonna save  ./features/squeezenet/Allaple.A/00f971e69aae4ccd38f7edae05aa5314.pt


  2%|██▏                                                                                                        | 2/100 [00:01<01:06,  1.47it/s]

gonna save  ./features/vgg16/Allaple.A/065a849200a7b1dc5f5474a187f41779.pt
gonna save  ./features/resnet50/Allaple.A/065a849200a7b1dc5f5474a187f41779.pt
gonna save  ./features/inception_v3/Allaple.A/065a849200a7b1dc5f5474a187f41779.pt
gonna save  ./features/squeezenet/Allaple.A/065a849200a7b1dc5f5474a187f41779.pt


...

gonna save  ./features/vgg16/C2LOP.P/5aug07c5696e49b0b25b34fcf52d32cc764f.pt
gonna save  ./features/resnet50/C2LOP.P/5aug07c5696e49b0b25b34fcf52d32cc764f.pt
gonna save  ./features/inception_v3/C2LOP.P/5aug07c5696e49b0b25b34fcf52d32cc764f.pt
gonna save  ./features/squeezenet/C2LOP.P/5aug07c5696e49b0b25b34fcf52d32cc764f.pt


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [01:35<00:00,  1.18it/s]

gonna save  ./features/vgg16/C2LOP.P/05cb7e9608214cf7a794f6809614ba2b.pt
gonna save  ./features/resnet50/C2LOP.P/05cb7e9608214cf7a794f6809614ba2b.pt
gonna save  ./features/inception_v3/C2LOP.P/05cb7e9608214cf7a794f6809614ba2b.pt
gonna save  ./features/squeezenet/C2LOP.P/05cb7e9608214cf7a794f6809614ba2b.pt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:35<00:00,  1.04it/s]

gonna save  ./features/vgg16/C2LOP.P/4aug053ee68fd630421b341907a961306e2c.pt
gonna save  ./features/resnet50/C2LOP.P/4aug053ee68fd630421b341907a961306e2c.pt
gonna save  ./features/inception_v3/C2LOP.P/4aug053ee68fd630421b341907a961306e2c.pt
gonna save  ./features/squeezenet/C2LOP.P/4aug053ee68fd630421b341907a961306e2c.pt
